In [96]:
import os
import argparse

MemSize = 1000  # Memory size, though still 32-bit addressable

class InsMem(object):
    def __init__(self, name, ioDir):
        self.id = name
        with open(ioDir + "/Sample_Testcases_SS/input/testcase1/imem.txt") as im:
            # Read the instruction memory file and store it as 8-bit binary strings
            self.IMem = [data.strip() for data in im.readlines()]

    def readInstr(self, ReadAddress):
        # Convert the byte address to an index for 32-bit instruction (4 bytes per instruction)
        index = ReadAddress // 4 * 4  # Ensure the address is aligned to 4 bytes

        # Check if the address is within bounds
        if index + 3 < len(self.IMem):
            # Concatenate four 8-bit lines to create a 32-bit instruction
            instruction = self.IMem[index] + self.IMem[index + 1] + self.IMem[index + 2] + self.IMem[index + 3]
      
            # Convert the 32-bit binary string to a hexadecimal value
            return hex(int(instruction, 2))
        else:
            return None  # Return None if the address is out of range

In [97]:
ioDir = "/Users/jianxiongshen/Downloads/ECE6913ComputerArchitecture/project_related"  # Replace with the actual path to the folder containing 'input/imem.txt'
instruction_memory = InsMem("IMem_Instance", ioDir)

# Address is a multiple of 4 (e.g., 0, 4, 8, etc.)
print(instruction_memory.readInstr(0) )
instruction_memory.readInstr(16) 

0x83


'0x302423'

In [98]:
class DataMem(object):
    def __init__(self, name, ioDir):
        self.id = name
        self.ioDir = ioDir
        with open(ioDir + "/Sample_Testcases_SS/input/testcase1/dmem.txt") as dm:
            self.DMem = [data.strip() for data in dm.readlines()]

    def ensure_memory_size(self, min_size):
        # Extend the memory with zeros if the required index exceeds current size
        if min_size > len(self.DMem):
            self.DMem.extend(['00000000'] * (min_size - len(self.DMem)))

    def readInstr(self, ReadAddress):
        index = ReadAddress // 4 * 4  
        self.ensure_memory_size(index + 4)  
        data_word = self.DMem[index] + self.DMem[index + 1] + self.DMem[index + 2] + self.DMem[index + 3]
        return hex(int(data_word, 2))

    def writeDataMem(self, Address, WriteData):
        index = Address // 4 * 4  
        self.ensure_memory_size(index + 4)  # Ensure memory is large enough
        data_word = format(WriteData, '032b')
        self.DMem[index] = data_word[0:8]
        self.DMem[index + 1] = data_word[8:16]
        self.DMem[index + 2] = data_word[16:24]
        self.DMem[index + 3] = data_word[24:32]

    def outputDataMem(self):
        resPath = self.ioDir + "/" + self.id + "_DMEMResult.txt"
        with open(resPath, "w") as rp:
            rp.writelines([str(data) + "\n" for data in self.DMem])

In [99]:
ioDir = "/Users/jianxiongshen/Downloads/ECE6913ComputerArchitecture/project_related"  # Replace with the actual path to the folder containing 'input/imem.txt' # Your project directory
dmem = DataMem("Data", ioDir)


# Read first word (4 bytes starting at address 0)
word0 = dmem.readInstr(0)
print(f"Word at address 0: {word0}")  # Output: 0x1020304

# Read second word (4 bytes starting at address 4)
word1 = dmem.readInstr(4)
print(f"Word at address 4: {word1}")  # Output: 0xaaf0ff

# Example 2: Writing data
# Write value 0x12345678 to address 0
dmem.writeDataMem(0, 0x12345678)

# Read back the written value
updated_word = dmem.readInstr(0)
print(f"Updated word at address 0: {updated_word}")  # Output: 0x12345678

# Example 3: Output memory contents to file
dmem.outputDataMem()  # Creates Data_DMEMResult.txt with the current memory contents




Word at address 0: 0x5
Word at address 4: 0x3
Updated word at address 0: 0x12345678


In [100]:
class RegisterFile(object):
    def __init__(self, ioDir):
        self.outputFile = ioDir + "/RFResult.txt"
        self.Registers = [0x0 for i in range(32)]  # Initialize 32 registers to 0
    
    def readRF(self, Reg_addr):
        # Read the value of the register at Reg_addr
        if 0 <= Reg_addr < 32:
            return self.Registers[Reg_addr]
        else:
            raise IndexError("Register address out of bounds")

    def writeRF(self, Reg_addr, Wrt_reg_data):
        # Write data to the register at Reg_addr if it's not x0
        if 1 <= Reg_addr < 32:  # Register x0 must always remain 0
            self.Registers[Reg_addr] = Wrt_reg_data
    
    def outputRF(self, cycle):
        # Prepare the output format for the register file
        op = ["-" * 70 + "\n", "State of RF after executing cycle:" + str(cycle) + "\n"]
        op.extend([f"{val & 0xFFFFFFFF:032b}\n" for val in self.Registers])  # Mask and format each register value as a 32-bit binary string
        # Open the file and write (or append) based on the cycle count
        perm = "w" if cycle == 0 else "a"
        with open(self.outputFile, perm) as file:
            file.writelines(op)


In [101]:
# Create a RegisterFile instance
ioDir = "/Users/jianxiongshen/Downloads/ECE6913ComputerArchitecture/project_related"
rf = RegisterFile(ioDir)

# Example 1: Write some values to registers
rf.writeRF(1, 0x12345678)  # Write to x1
rf.writeRF(2, 0xABCDEF00)  # Write to x2
rf.writeRF(0, 0xFFFFFFFF)  # Try to write to x0 (should be ignored)

# Example 2: Read values from registers
x1_value = rf.readRF(1)
x2_value = rf.readRF(2)
x0_value = rf.readRF(0)

print(f"x1 value: {hex(x1_value)}")  # Should print: 0x12345678
print(f"x2 value: {hex(x2_value)}")  # Should print: 0xabcdef00
print(f"x0 value: {hex(x0_value)}")  # Should print: 0x0 (always zero)

# Example 3: Output the register file state to RFResult.txt
rf.outputRF(0)  # Output state after cycle 0

# Make another change and output again
rf.writeRF(3, 0x99999999)
rf.outputRF(1)  # Output state after cycle 1

x1 value: 0x12345678
x2 value: 0xabcdef00
x0 value: 0x0


In [102]:
class State(object):
    def __init__(self):
        self.IF = {"nop": False, "PC": 0}
        self.ID = {"nop": False, "Instr": 0}
        self.EX = {"nop": False, "Read_data1": 0, "Read_data2": 0, "Imm": 0, "Rs": 0, "Rt": 0, "Wrt_reg_addr": 0, "is_I_type": False, "rd_mem": 0, 
                   "wrt_mem": 0, "alu_op": 0, "wrt_enable": 0, "branch": False}
        self.MEM = {"nop": False, "ALUresult": 0, "Store_data": 0, "Rs": 0, "Rt": 0, "Wrt_reg_addr": 0, "rd_mem": 0, 
                   "wrt_mem": 0, "wrt_enable": 0}
        self.WB = {"nop": False, "Wrt_data": 0, "Rs": 0, "Rt": 0, "Wrt_reg_addr": 0, "wrt_enable": 0}

class Core(object):
    def __init__(self, ioDir, imem, dmem):
        self.myRF = RegisterFile(ioDir)
        self.cycle = 0
        self.halted = False
        self.ioDir = ioDir
        self.state = State()
        self.nextState = State()
        self.ext_imem = imem
        self.ext_dmem = dmem
        self.instructionCount = 0

In [103]:
class SingleStageCore(Core):
    def __init__(self, ioDir, imem, dmem):
        super(SingleStageCore, self).__init__(ioDir, imem, dmem)
        self.opFilePath = os.path.join(ioDir,  "StateResult_SS.txt")

    def IF(self):
        self.state.ID["Instr"] = self.ext_imem.readInstr(self.state.IF["PC"])
        if self.state.ID["Instr"] is not None:
            opcode = self.state.ID["Instr"][-7:]  
            
            if opcode == "1111111":  # nop instruction
                self.nextState.IF["PC"] = self.state.IF["PC"]
                self.nextState.IF["nop"] = True

            else:
                self.nextState.IF["nop"] = False
                self.nextState.IF["PC"] = self.state.IF["PC"] + 4;
                self.state.ID["nop"] = False  
                self.instructionCount += 1 

        else:
            self.state.IF["nop"] = True  # No valid instruction, set nop to true

            
    def ID(self):
        # Decode the instruction from the ID stage
        instruction = self.state.ID["Instr"]
        # print(instruction)
        if instruction is None:
            self.halted = True
            return
        
        instruction = int(instruction, 16) if isinstance(instruction, str) else instruction
        # print(instruction)
        
        # Extract fields from the instruction
        opcode = instruction & 0x7F
        self.state.ID["Instr"] = instruction  # Store the instruction in the state

        # rs --> rs1, rt --> rs2
        if opcode == 0x33:  # R-type instructions (e.g., ADD)
            funct3 = (instruction >> 12) & 0x7
            print(funct3)
            funct7 = (instruction >> 25) & 0x7F
            rd = (instruction >> 7) & 0x1F
            rs1 = (instruction >> 15) & 0x1F
            rs2 = (instruction >> 20) & 0x1F
            self.state.EX["Read_data1"] = self.myRF.readRF(rs1)  # Read data from register file
            self.state.EX["Read_data2"] = self.myRF.readRF(rs2)  # Read data from register file
            self.state.EX["Imm"] = 0  # No immediate for R-type
            self.state.EX["Rs"] = rs1
            self.state.EX["Rt"] = rs2
            self.state.EX["Wrt_reg_addr"] = rd  # Destination register
            self.state.EX["rd_mem"] = False
            self.state.EX["wrt_mem"] = False
            self.state.EX["is_I_type"] = False  # R-type instruction

            ALUmapping = {
                0: "0010",  # ADD (also used for SUB, but SUB is distinguished by funct7)
                4: "0011",  # XOR
                6: "0001",  # OR
                7: "0000",  # AND
            }

            # Handle funct7 for distinguishing ADD and SUB
            if funct3 == 0:  # Check if funct3 is 0 (binary 000)
                if funct7 == 0x00:
                    self.state.EX["alu_op"] = "0010"  # ADD
                elif funct7 == 0x20:
                    self.state.EX["alu_op"] = "0110"  # SUB
            else:
                self.state.EX["alu_op"] = ALUmapping[funct3]  # Use funct3 as an integer index

            self.state.EX["wrt_enable"] = True
       
            
        
        elif opcode == 0x13:  # I-type instructions (e.g., ADDI, XORI, ORI, ANDI)
            funct3 = (instruction >> 12) & 0x7
            rd = (instruction >> 7) & 0x1F
            rs1 = (instruction >> 15) & 0x1F
            imm = (instruction >> 20) & 0xFFF  # Immediate value (bits 20-31)

            # Sign-extend the immediate value
            if imm & 0x800:  # Check if the sign bit (bit 11) is set
                imm |= 0xFFFFF000  # Extend to 32 bits (for 12-bit immediate)

            self.state.EX["Wrt_reg_addr"] = rd
            self.state.EX["Rs"] = rs1
            self.state.EX["Read_data1"] = self.myRF.readRF(rs1)  # Read data from register file
            self.state.EX["Read_data2"] = 0
            self.state.EX["Rt"] = 0
            self.state.EX["Imm"] = imm  # Store the sign-extended immediate value
            self.state.EX["rd_mem"] = False
            self.state.EX["wrt_mem"] = False
            self.state.EX["is_I_type"] = True  # Mark as I-type instruction
            self.state.EX["wrt_enable"] = True

            # Mapping funct3 to ALU operations
            ALUmapping = {
                0x0: "0010",  # ADDI
                0x4: "0011",  # XORI
                0x6: "0001",  # ORI
                0x7: "0000",  # ANDI
            }

            self.state.EX["alu_op"] = ALUmapping[funct3]  # Set ALU operation based on funct3

      

        elif opcode == 0x03:  # LOAD instructions (I-type)
            funct3 = (instruction >> 12) & 0x7  # Extract funct3 (bits 12-14)
            rd = (instruction >> 7) & 0x1F  # Destination register
            rs1 = (instruction >> 15) & 0x1F  # Source register 1
            imm = (instruction >> 20) & 0xFFF  # Immediate value (bits 20-31)

            # Sign-extend the immediate value
            if imm & 0x800:  # Check if the sign bit (bit 11) is set
                imm |= 0xFFFFF000  # Extend to 32 bits (for 12-bit immediate)

            self.state.EX["Wrt_reg_addr"] = rd
            self.state.EX["Rs"] = rs1
            self.state.EX["Read_data1"] = self.myRF.readRF(rs1)  # Read data from register file
            self.state.EX["Read_data2"] = 0
            self.state.EX["Rt"] = 0
            self.state.EX["Imm"] = imm  # Store the sign-extended immediate value
            self.state.EX["rd_mem"] = True  # Indicate that this is a read memory operation
            self.state.EX["wrt_mem"] = False
            self.state.EX["is_I_type"] = True  # Mark as I-type instruction
            
            # to support addi, andi, subi, ori, xori
            ALUmapping = {
                "0000": "0010",  # ADD
                "0001": "0110",  # SUB
                "1110": "0000",  # AND
                "1100": "0001",  # OR
                "1000": "0011",  # XOR
            }
            
            self.state.EX["wrt_enable"] = True
            self.state.EX["alu_op"] = ALUmapping[format(funct3, '04b')]  # Convert funct3 to 4-bit binary string
        
        elif opcode == 0x6F:  # JAL instruction
            rd = (instruction >> 7) & 0x1F
            # Extract immediate using the specified bit positions and sign extension
            imm = ((instruction & 0x80000000) >> 11) | \
                ((instruction & 0x7E000000) >> 20) | \
                ((instruction & 0x100000) >> 9) | \
                ((instruction & 0xFF000))
            # Sign-extend the immediate value
            if imm & 0x80000:  # Check if the sign bit (20th bit) is set
                imm |= 0xFFF00000  # Sign-extend to 32 bits           
         
            self.instructionCount += 1
            self.state.EX["funct3"] = "111"
            self.state.EX["Wrt_reg_addr"] = rd
            self.state.EX["Read_data1"] = 0
            self.state.EX["Read_data2"] = 0
            self.state.EX["Imm"] = imm
            self.state.EX["rd_mem"] = False
            self.state.EX["wrt_mem"] = False
            self.state.EX["is_I_type"] = False
            self.state.EX["wrt_enable"] = True
            self.state.EX["branch"] = True
            self.state.EX["alu_op"] = "0010"  
        

        elif opcode == 0x63:  # B-type instructions
            funct3 = (instruction >> 12) & 0x7
            rs1 = (instruction >> 15) & 0x1F
            rs2 = (instruction >> 20) & 0x1F
            # Extract immediate using the specified bit positions and sign extension
            imm = ((instruction & 0x80000000) >> 19) | \
                ((instruction & 0x80) << 4) | \
                ((instruction & 0x7E000000) >> 20) | \
                ((instruction & 0xF00) >> 7)
            # Sign-extend the immediate value
            if imm & 0x1000:  # Check if the sign bit (12th bit) is set
                imm |= 0xFFFFE000  # Sign-extend to 32 bits

            read_data1 = self.myRF.readRF(rs1)
            read_data2 = self.myRF.readRF(rs2)

            self.state.EX["funct3"] = funct3
            self.state.EX["Wrt_reg_addr"] = 0
            self.state.EX["Rs"] = rs1
            self.state.EX["Read_data1"] = read_data1
            self.state.EX["Rt"] = rs2
            self.state.EX["Read_data2"] = read_data2
            self.state.EX["Imm"] = imm
            self.state.EX["rd_mem"] = False
            self.state.EX["wrt_mem"] = False
            self.state.EX["is_I_type"] = False
            self.state.EX["wrt_enable"] = False
            self.state.EX["branch"] = True
            self.state.EX["alu_op"] = "0110"  

        elif opcode == 0x23:  # S-type instructions (e.g., SW)
            funct3 = (instruction >> 12) & 0x7
            rs1 = (instruction >> 15) & 0x1F
            rs2 = (instruction >> 20) & 0x1F
            # Extract immediate using the specified bit positions and sign extension
            imm = ((instruction & 0xFE000000) >> 20) | \
                ((instruction & 0xF80) >> 7)
            # Sign-extend the immediate value
            if imm & 0x800:  # Check if the sign bit (11th bit) is set
                imm |= 0xFFFFF000  # Sign-extend to 32 bits

            read_data1 = self.myRF.readRF(rs1)
            read_data2 = self.myRF.readRF(rs2)

            self.state.EX["funct3"] = funct3
            self.state.EX["Wrt_reg_addr"] = 0
            self.state.EX["Rs"] = rs1
            self.state.EX["Read_data1"] = read_data1
            self.state.EX["Rt"] = rs2
            self.state.EX["Read_data2"] = read_data2
            self.state.EX["Imm"] = imm
            self.state.EX["rd_mem"] = False
            self.state.EX["wrt_mem"] = True
            self.state.EX["is_I_type"] = True
            self.state.EX["wrt_enable"] = False
            self.state.EX["alu_op"] = "0010" 

        print("instruction is: ", bin(self.state.ID["Instr"]), "state.EX is: ", self.state.EX)


    def EX(self):
        if not self.state.EX["nop"]:
            # Determine the second ALU input
            if self.state.EX["is_I_type"]:
                # For I-type instructions, the second ALU input is the immediate value
                # convert unsigned to signed
                ALU2 = self.state.EX["Imm"] if self.state.EX["Imm"] < 0x8000 else self.state.EX["Imm"] - 0x10000
            else:
                # For R-type instructions, the second ALU input is the second register value
                ALU2 = self.state.EX["Read_data2"]
            
            if self.state.EX["alu_op"] == "0010":  # ADD or ADDI
                self.state.MEM["ALUresult"] = self.state.EX["Read_data1"] + ALU2
            elif self.state.EX["alu_op"] == "0110":  # SUB
                self.state.MEM["ALUresult"] = self.state.EX["Read_data1"] - ALU2
            elif self.state.EX["alu_op"] == "0000":  # AND or ANDI
                self.state.MEM["ALUresult"] = self.state.EX["Read_data1"] & ALU2
            elif self.state.EX["alu_op"] == "0001":  # OR or ORI
                self.state.MEM["ALUresult"] = self.state.EX["Read_data1"] | ALU2
            elif self.state.EX["alu_op"] == "0011":  # XOR or XORI
                self.state.MEM["ALUresult"] = self.state.EX["Read_data1"] ^ ALU2

            if self.state.EX["branch"]:
                if self.state.EX["funct3"] == 0x0 and self.state.MEM["ALUresult"] == 0:  # beq
                    self.nextState.IF["PC"] = self.state.IF["PC"] + self.state.EX["Imm"]
                    self.nextState.IF["nop"] = False
                    self.state.MEM["nop"] = True
                elif self.state.EX["funct3"] == 0x1 and self.state.MEM["ALUresult"] != 0:  # bne
                    self.nextState.IF["PC"] = self.state.IF["PC"] + self.state.EX["Imm"]
                    self.nextState.IF["nop"] = False
                    self.state.MEM["nop"] = True
                elif self.state.EX["funct3"] == 0x7: 
                    self.nextState.IF["nop"] = False
                    self.state.MEM["ALUresult"] = self.state.IF["PC"] + 4
                    self.nextState.IF["PC"] = self.state.IF["PC"] + self.state.EX["Imm"]


            self.state.MEM["rd_mem"] = self.state.EX["rd_mem"]
            self.state.MEM["wrt_mem"] = self.state.EX["wrt_mem"]

        print("ALUResult is: ", self.state.MEM["ALUresult"])
    
    def MEM(self):
        self.state.WB["nop"] = self.state.MEM["nop"]
        if not self.state.MEM["nop"]:
            # Handle memory read operation if required
            if self.state.MEM["rd_mem"]:
                self.state.MEM["Store_data"] = self.ext_dmem.readInstr(self.state.MEM["ALUresult"])
            
            # Handle memory write operation if required
            if self.state.MEM["wrt_mem"]:
                self.ext_dmem.writeDataMem(self.state.MEM["ALUresult"], self.state.EX["Read_data2"])

            # Pass the ALU result to the write-back stage
            self.state.WB["ALUresult"] = self.state.MEM["ALUresult"]  # Ensure ALU result is passed to WB stage

            # Update write-back control signals
            self.state.MEM["Wrt_reg_addr"] = self.state.EX["Wrt_reg_addr"]
            self.state.WB["Wrt_reg_addr"] = self.state.MEM["Wrt_reg_addr"]
            self.state.WB["wrt_enable"] = self.state.EX["wrt_enable"]
        else:
            self.state.WB["nop"] = True



        print("instruction is: ", bin(self.state.ID["Instr"]), "state.MEM is: ", self.state.MEM)


    def WB(self):
        if not self.state.WB["nop"] and self.state.WB["wrt_enable"]:
            print("Wrt_reg_addr and Store_data are",(self.state.WB["Wrt_reg_addr"], self.state.MEM["Store_data"]))  # Debug print
            if self.state.EX["rd_mem"]:
               
                store_data_value = int(self.state.MEM["Store_data"], 16)
                self.myRF.writeRF(self.state.WB["Wrt_reg_addr"], store_data_value)
            else:
                self.myRF.writeRF(self.state.WB["Wrt_reg_addr"], self.state.MEM["ALUresult"])


    def step(self):
        self.IF()
        self.ID()
        self.EX()
        self.MEM()
        self.WB()


        if self.state.IF["nop"]:
            self.halted = True
    
        # Dump RF and print state
        self.myRF.outputRF(self.cycle)  # Dump Register File
        self.printState(self.nextState, self.cycle)  # Print states after executing the cycle
        
        # Prepare for the next cycle
        self.state = self.nextState  # Update the current state
        self.cycle += 1

    def printState(self, state, cycle):
        printstate = ["-"*70 + "\n", "State after executing cycle: " + str(cycle) + "\n"]
        printstate.append("IF.PC: " + str(state.IF["PC"]) + "\n")
        printstate.append("IF.nop: " + str(state.IF["nop"]) + "\n")
        
        perm = "w" if cycle == 0 else "a"
        with open(self.opFilePath, perm) as wf:
            wf.writelines(printstate)

        


In [104]:
ioDir = "/Users/jianxiongshen/Downloads/ECE6913ComputerArchitecture/project_related"
print("IO Directory:", ioDir)

imem = InsMem("Imem", ioDir)
dmem_ss = DataMem("SS", ioDir)

ssCore = SingleStageCore(ioDir, imem, dmem_ss)


while(True):
    if not ssCore.halted:
        ssCore.step()

    if ssCore.halted:
        break


# dump SS and FS data mem.
dmem_ss.outputDataMem()


IO Directory: /Users/jianxiongshen/Downloads/ECE6913ComputerArchitecture/project_related
instruction is:  0b10000011 state.EX is:  {'nop': False, 'Read_data1': 0, 'Read_data2': 0, 'Imm': 0, 'Rs': 0, 'Rt': 0, 'Wrt_reg_addr': 1, 'is_I_type': True, 'rd_mem': True, 'wrt_mem': False, 'alu_op': '0010', 'wrt_enable': True, 'branch': False}
ALUResult is:  0
instruction is:  0b10000011 state.MEM is:  {'nop': False, 'ALUresult': 0, 'Store_data': '0x5', 'Rs': 0, 'Rt': 0, 'Wrt_reg_addr': 1, 'rd_mem': True, 'wrt_mem': False, 'wrt_enable': 0}
Wrt_reg_addr and Store_data are (1, '0x5')
instruction is:  0b10000000000000100000011 state.EX is:  {'nop': False, 'Read_data1': 0, 'Read_data2': 0, 'Imm': 4, 'Rs': 0, 'Rt': 0, 'Wrt_reg_addr': 2, 'is_I_type': True, 'rd_mem': True, 'wrt_mem': False, 'alu_op': '0010', 'wrt_enable': True, 'branch': False}
ALUResult is:  4
instruction is:  0b10000000000000100000011 state.MEM is:  {'nop': False, 'ALUresult': 4, 'Store_data': '0x3', 'Rs': 0, 'Rt': 0, 'Wrt_reg_addr': 

TypeError: 'NoneType' object cannot be interpreted as an integer

In [ ]:
class PipelineState(object):
    def __init__(self):
        # Initialize pipeline registers (PR) between stages
        self.IF_ID = None
        self.ID_EX = None
        self.EX_MEM = None
        self.MEM_WB = None

class PipelinedCore(Core):
    def __init__(self, ioDir, imem, dmem):
        super(PipelinedCore, self).__init__(ioDir, imem, dmem)
        self.pipelineState = PipelineState()
        self.stalls = 0
        self.branches_taken = 0

    def IF(self):
        # Fetch instruction and update IF/ID pipeline register
        # Handle PC and branching
        pass

    def ID(self):
        # Decode instruction and update ID/EX pipeline register
        # Handle hazard detection and potential stalls
        pass

    def EX(self):
        # Execute instruction, calculate ALU ops, and update EX/MEM pipeline register
        # Implement forwarding paths here
        pass

    def MEM(self):
        # Access memory if needed and update MEM/WB pipeline register
        pass

    def WB(self):
        # Write back to registers
        pass

    def step(self):
        # Simulate one cycle of the pipeline
        self.WB()
        self.MEM()
        self.EX()
        self.ID()
        self.IF()
        # Update cycle count, handle pipeline state transitions
        self.cycle += 1

# Usage
ssCore = PipelinedCore(ioDir, imem, dmem_ss)
while not ssCore.halted:
    ssCore.step()